In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob
import seaborn as sns
from scipy.stats import kruskal
import scikit_posthocs as sp
from scipy.stats import mannwhitneyu
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()
from google.cloud import bigquery_storage_v1

## Read patients_Notes_EHR

In [3]:
%%bigquery Train_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2015 and order_year<2022

Query is running:   0%|          |

Downloading:   0%|          |

In [4]:
%%bigquery Val_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2022 and order_year<2023

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
%%bigquery Test_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2023 

Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
%%bigquery Notes_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.EDProviderNotes`

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
Train_set_df_note=pd.merge(Train_set_df,Notes_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Train_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Val_set_df_note=pd.merge(Val_set_df,Notes_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Val_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Test_set_df_note=pd.merge(Test_set_df,Notes_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Test_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

(25138, 3)

In [8]:
Train_set_df_note=Train_set_df_note[Train_set_df_note.deid_note_text.notnull()]
Train_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Val_set_df_note=Val_set_df_note[Val_set_df_note.deid_note_text.notnull()]
Val_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Test_set_df_note=Test_set_df_note[Test_set_df_note.deid_note_text.notnull()]
Test_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

(25138, 3)

In [9]:
Train_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Train_set_df_note.shape

Val_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Val_set_df_note.shape

Test_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Test_set_df_note.shape


(26247, 113)

In [10]:
Train_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Train_set_df_note['blood_culture_order_datetime'])
Train_set_df_note['notedatetime'] = pd.to_datetime(Train_set_df_note['notedatetime'])
Train_set_df_note['time-diff']=(Train_set_df_note['blood_culture_order_datetime']-Train_set_df_note['notedatetime']).dt.total_seconds()/3600
Train_set_df_note['time-diff']=Train_set_df_note['time-diff'].astype(int)
Train_set_df_note=Train_set_df_note[Train_set_df_note['time-diff']>=-24]


Val_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Val_set_df_note['blood_culture_order_datetime'])
Val_set_df_note['notedatetime'] = pd.to_datetime(Val_set_df_note['notedatetime'])
Val_set_df_note['time-diff']=(Val_set_df_note['blood_culture_order_datetime']-Val_set_df_note['notedatetime']).dt.total_seconds()/3600
Val_set_df_note['time-diff']=Val_set_df_note['time-diff'].astype(int)
Val_set_df_note=Val_set_df_note[Val_set_df_note['time-diff']>=-24]

Test_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Test_set_df_note['blood_culture_order_datetime'])
Test_set_df_note['notedatetime'] = pd.to_datetime(Test_set_df_note['notedatetime'])
Test_set_df_note['time-diff']=(Test_set_df_note['blood_culture_order_datetime']-Test_set_df_note['notedatetime']).dt.total_seconds()/3600
Test_set_df_note['time-diff']=Test_set_df_note['time-diff'].astype(int)
Test_set_df_note=Test_set_df_note[Test_set_df_note['time-diff']>=-24]


In [11]:
Train_set_df_note['time-diff']=(Train_set_df_note['blood_culture_order_datetime']-Train_set_df_note['notedatetime']).dt.total_seconds().abs()
Val_set_df_note['time-diff']=(Val_set_df_note['blood_culture_order_datetime']-Val_set_df_note['notedatetime']).dt.total_seconds().abs()
Test_set_df_note['time-diff']=(Test_set_df_note['blood_culture_order_datetime']-Test_set_df_note['notedatetime']).dt.total_seconds().abs()


Train_set_df_note = Train_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')
Val_set_df_note = Val_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')
Test_set_df_note = Test_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')


In [13]:
Test_set_df_note['Label']=(Test_set_df_note['positive_blood_culture']| Test_set_df_note['positive_blood_culture_in_week'])
Train_set_df_note['Label']=(Train_set_df_note['positive_blood_culture']| Train_set_df_note['positive_blood_culture_in_week'])
Val_set_df_note['Label']=(Val_set_df_note['positive_blood_culture']| Val_set_df_note['positive_blood_culture_in_week'])

In [14]:
EHR=[col for col in Test_set_df_note.columns.values if 'min' in col]
EHR.extend([col for col in Test_set_df_note.columns.values if 'max' in col])
EHR.extend([col for col in Test_set_df_note.columns.values if 'avg' in col])
EHR.extend([col for col in Test_set_df_note.columns.values if 'median' in col])
EHR

['min_heartrate',
 'min_resprate',
 'min_temp',
 'min_sysbp',
 'min_diasbp',
 'min_wbc',
 'min_neutrophils',
 'min_lymphocytes',
 'min_hgb',
 'min_plt',
 'min_na',
 'min_hco3',
 'min_bun',
 'min_cr',
 'min_lactate',
 'min_procalcitonin',
 'max_heartrate',
 'max_resprate',
 'max_temp',
 'max_sysbp',
 'max_diasbp',
 'max_wbc',
 'max_neutrophils',
 'max_lymphocytes',
 'max_hgb',
 'max_plt',
 'max_na',
 'max_hco3',
 'max_bun',
 'max_cr',
 'max_lactate',
 'max_procalcitonin',
 'avg_heartrate',
 'avg_resprate',
 'avg_temp',
 'avg_sysbp',
 'avg_diasbp',
 'avg_wbc',
 'avg_neutrophils',
 'avg_lymphocytes',
 'avg_hgb',
 'avg_plt',
 'avg_na',
 'avg_hco3',
 'avg_bun',
 'avg_cr',
 'avg_lactate',
 'avg_procalcitonin',
 'median_heartrate',
 'median_resprate',
 'median_temp',
 'median_sysbp',
 'median_diasbp',
 'median_wbc',
 'median_neutrophils',
 'median_lymphocytes',
 'median_hgb',
 'median_plt',
 'median_na',
 'median_hco3',
 'median_bun',
 'median_cr',
 'median_lactate',
 'median_procalcitonin']

## Select top 3 most relevent chunk

In [89]:
Train_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Train_set_df_note.shape

Val_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Val_set_df_note.shape

Test_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Test_set_df_note.shape


(26247, 113)

In [90]:
Train_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Train_set_df_note['blood_culture_order_datetime'])
Train_set_df_note['notedatetime'] = pd.to_datetime(Train_set_df_note['notedatetime'])
Train_set_df_note['time-diff']=(Train_set_df_note['blood_culture_order_datetime']-Train_set_df_note['notedatetime']).dt.total_seconds().abs()

Val_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Val_set_df_note['blood_culture_order_datetime'])
Val_set_df_note['notedatetime'] = pd.to_datetime(Val_set_df_note['notedatetime'])
Val_set_df_note['time-diff']=(Val_set_df_note['blood_culture_order_datetime']-Val_set_df_note['notedatetime']).dt.total_seconds().abs()

Test_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Test_set_df_note['blood_culture_order_datetime'])
Test_set_df_note['notedatetime'] = pd.to_datetime(Test_set_df_note['notedatetime'])
Test_set_df_note['time-diff']=(Test_set_df_note['blood_culture_order_datetime']-Test_set_df_note['notedatetime']).dt.total_seconds().abs()

In [91]:
Train_set_df_note = Train_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')
Val_set_df_note = Val_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')
Test_set_df_note = Test_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')


In [92]:
Test_set_df_note.shape

(25138, 114)

In [93]:
Train_set_df_note['doc_id'] = Train_set_df_note.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime']).ngroup() + 1
Val_set_df_note['doc_id'] = Val_set_df_note.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime']).ngroup() + 1
Test_set_df_note['doc_id'] = Test_set_df_note.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime']).ngroup() + 1

In [94]:
def chunk_text_by_words(df, text_col='deid_note_text', id_col='doc_id', max_words=512):
    import re
    chunked_rows = []

    for _, row in df.iterrows():
        text = str(row[text_col])
        
        # Split into sentences using regex for robustness
        sentences = re.split(r'(?<=[.!?]) +|\n+', text)
        
        chunks = []
        current_chunk = []
        current_word_count = 0
        chunk_idx = 1

        for sentence in sentences:
            sentence = sentence.strip()
            word_count = len(sentence.split())

            # If adding this sentence exceeds max_words, start new chunk
            if current_word_count + word_count > max_words:
                if current_chunk:
                    chunk_text = ' '.join(current_chunk)
                    new_row = row.copy()
                    new_row[text_col] = chunk_text
                    new_row[id_col] = f"{row[id_col]}_chunk{chunk_idx}"
                    chunked_rows.append(new_row)
                    chunk_idx += 1
                current_chunk = [sentence]
                current_word_count = word_count
            else:
                current_chunk.append(sentence)
                current_word_count += word_count

        # Add the last chunk
        if current_chunk:
            chunk_text = ' '.join(current_chunk)
            new_row = row.copy()
            new_row[text_col] = chunk_text
            new_row[id_col] = f"{row[id_col]}_chunk{chunk_idx}"
            chunked_rows.append(new_row)

    return pd.DataFrame(chunked_rows)
Train_set_df_note_chunked = chunk_text_by_words(Train_set_df_note)
Val_set_df_note_chunked = chunk_text_by_words(Val_set_df_note)
Test_set_df_note_chunked = chunk_text_by_words(Test_set_df_note)


In [ ]:
Train_set_df_note_chunked['doc_id_chunks']=Train_set_df_note_chunked['doc_id']
Train_set_df_note_chunked['doc_id']=Train_set_df_note_chunked['doc_id'].apply(lambda x:x.split('_chunk')[0])

Val_set_df_note_chunked['doc_id_chunks']=Val_set_df_note_chunked['doc_id']
Val_set_df_note_chunked['doc_id']=Val_set_df_note_chunked['doc_id'].apply(lambda x:x.split('_chunk')[0])

Test_set_df_note_chunked['doc_id_chunks']=Test_set_df_note_chunked['doc_id']
Test_set_df_note_chunked['doc_id']=Test_set_df_note_chunked['doc_id'].apply(lambda x:x.split('_chunk')[0])

In [96]:
Train_set_df_note_chunked.to_csv('Train_set_df_ED_notes_chunked.csv',index=False)
Val_set_df_note_chunked.to_csv('Val_set_df_ED_notes_chunked.csv',index=False)
Test_set_df_note_chunked.to_csv('Test_set_df_ED_notes_chunked.csv',index=False)

## embdeing of each chunk has been extracted from fatemeh-nick-culturx machine